#### Задание 1
Сгенерировать последовательности, которые бы состояли из цифр (от 0 до 9) и задавались следующим образом:   
x - последовательность цифр   
y1 = x1, y(i) = x(i) + x(1). Если y(i) >= 10, то y(i) = y(i) - 10

**Задача:**
1. научить модель предсказывать y(i) по x(i)
2. пробовать RNN, LSTM, GRU

In [1]:
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

from sklearn.model_selection import train_test_split

##### 1. Генерация данных

In [2]:
def generate_sequence(length):
    x = np.random.randint(0, 10, length)
    y = np.zeros_like(x)
    y[0] = x[0]
    for i in range(1, length):
        y[i] = x[i] + x[0]
        if y[i] >= 10:
            y[i] -= 10
    return x, y

In [3]:
# Генерация набора данных
data_size = 10000
sequence_length = 10
X = []
Y = []
for _ in range(data_size):
    x, y = generate_sequence(sequence_length)
    X.append(x)
    Y.append(y)

X = np.array(X, dtype=int)
Y = np.array(Y, dtype=int)


In [21]:
# Посмотрим получившыйся набор данных Х
X

array([[7, 4, 6, ..., 7, 2, 9],
       [8, 1, 2, ..., 0, 0, 6],
       [1, 2, 0, ..., 9, 3, 9],
       ...,
       [1, 8, 4, ..., 4, 8, 5],
       [2, 6, 6, ..., 4, 9, 9],
       [0, 7, 3, ..., 3, 4, 7]], shape=(10000, 10))

In [5]:
# Посмотрим получившыйся набор данных Y
Y

array([[7, 1, 3, ..., 4, 9, 6],
       [8, 9, 0, ..., 8, 8, 4],
       [1, 3, 1, ..., 0, 4, 0],
       ...,
       [1, 9, 5, ..., 5, 9, 6],
       [2, 8, 8, ..., 6, 1, 1],
       [0, 7, 3, ..., 3, 4, 7]], shape=(10000, 10))

##### 2. Подготовка данных
Разделим данные на обучающую и тестовую выборки и преобразуем их в тензоры PyTorch.

In [6]:
# Разделение данных на обучающую и тестовую выборки
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# Преобразование данных в тензоры Pytorch
X_train = torch.Tensor(X_train).unsqueeze(-1) # Добавляем размерность для признаков
X_test = torch.Tensor(X_test).unsqueeze(-1)
Y_train = torch.Tensor(Y_train) 
Y_test = torch.Tensor(Y_test)

In [7]:
# Создадим DataLoader
train_dataset = TensorDataset(X_train, Y_train)
test_dataset = TensorDataset(X_test, Y_test)

batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

##### 3. Создание модели
Создадим модели RNN, LSTM и GRU

In [13]:
class LSTM_Model(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(LSTM_Model, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
    
    def forward(self, x):
        out, _ = self.lstm(x)
        out = self.fc(out[:, -1, :])
        return out

class GRU_Model(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(GRU_Model, self).__init__()
        self.gru = nn.GRU(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
    
    def forward(self, x):
        out, _ = self.gru(x)
        out = self.fc(out[:, -1, :])
        return out

class RNN_Model(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNN_Model, self).__init__()
        self.rnn = nn.RNN(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
    
    def forward(self, x):
        out, _ = self.rnn(x)
        out = self.fc(out[:, -1, :])
        return out

In [14]:
# Параметры модели
input_size = 1
hidden_size = 50
output_size = 10
num_layers = 1

In [15]:
# Инициализируем модели
rnn_model = RNN_Model(input_size, hidden_size, output_size)
lstm_model = LSTM_Model(input_size, hidden_size, output_size)
gru_model = GRU_Model(input_size, hidden_size, output_size) 

##### 4. Обучение моделей

In [16]:
# Функция потерь и оптимизатор
criterion = nn.MSELoss()
learning_rate = 0.001
num_epoch = 20

In [17]:
# Обучим модель
def train_model(model, train_loader, num_epochs=num_epoch):
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    for epoch in range(num_epoch):
        model.train()
        for X_batch, y_batch in train_loader:
            optimizer.zero_grad()
            outputs = model(X_batch)
            loss = criterion(outputs, y_batch)
            loss.backward()
            optimizer.step()
        print(f"Epoch [{epoch+1}/{num_epochs}], loss: {loss.item():.3f}")

print("Training RNN Model...")
train_model(rnn_model, train_loader)

print("Training LSTM Model...")
train_model(lstm_model, train_loader)

print("Training GRU Model...")
train_model(gru_model, train_loader)

Training RNN Model...
Epoch [1/20], loss: 8.894
Epoch [2/20], loss: 8.554
Epoch [3/20], loss: 8.421
Epoch [4/20], loss: 7.336
Epoch [5/20], loss: 8.525
Epoch [6/20], loss: 8.727
Epoch [7/20], loss: 9.019
Epoch [8/20], loss: 8.749
Epoch [9/20], loss: 8.566
Epoch [10/20], loss: 8.035
Epoch [11/20], loss: 8.626
Epoch [12/20], loss: 8.391
Epoch [13/20], loss: 8.735
Epoch [14/20], loss: 8.078
Epoch [15/20], loss: 8.748
Epoch [16/20], loss: 7.889
Epoch [17/20], loss: 8.185
Epoch [18/20], loss: 8.562
Epoch [19/20], loss: 8.137
Epoch [20/20], loss: 8.541
Training LSTM Model...
Epoch [1/20], loss: 7.972
Epoch [2/20], loss: 8.006
Epoch [3/20], loss: 8.188
Epoch [4/20], loss: 8.992
Epoch [5/20], loss: 7.527
Epoch [6/20], loss: 7.233
Epoch [7/20], loss: 7.320
Epoch [8/20], loss: 7.614
Epoch [9/20], loss: 6.827
Epoch [10/20], loss: 7.424
Epoch [11/20], loss: 7.845
Epoch [12/20], loss: 7.396
Epoch [13/20], loss: 7.095
Epoch [14/20], loss: 6.912
Epoch [15/20], loss: 6.606
Epoch [16/20], loss: 6.428
E

#### 5. Оценка моделей
Протестируем модели на тестовых данных.

In [18]:
def evaluate_model(model, test_loader):
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for X_batch, y_batch in test_loader:
            outputs = model(X_batch)
            loss = criterion(outputs, y_batch)
            total_loss +=loss.item()
    avg_loss = total_loss / len(test_loader)
    print(f"test loss: {avg_loss:.3f}")

print("Evaluating RNN Model...")
evaluate_model(rnn_model, test_loader)

print("Evaluating LSTM Model...")
evaluate_model(lstm_model, test_loader)

print("Evaluating GRU Model...")
evaluate_model(gru_model, test_loader)

Evaluating RNN Model...
test loss: 8.221
Evaluating LSTM Model...
test loss: 5.693
Evaluating GRU Model...
test loss: 0.463


#### 6. предсказание
Используем обученные модели для предсказания.

In [19]:
def predict_sequence(model, x):
    model.eval()
    with torch.no_grad():
        x = torch.tensor(x).unsqueeze(0).unsqueeze(-1)
        y_pred = model(x)
    return y_pred.squeeze().numpy()

# Пример предсказания
x_test = X_test[0].squeeze().numpy()
y_test = Y_test[0].numpy()

y_pred_rnn = predict_sequence(rnn_model, x_test)
y_pred_lstm = predict_sequence(lstm_model, x_test)
y_pred_gru = predict_sequence(gru_model, x_test)

print(f"True y: {y_test}")
print(f"RNN Predicted y: {y_pred_rnn}")
print(f"LSTM Predicted y: {y_pred_lstm}")
print(f"GRU Predicted y: {y_pred_gru}")

True y: [8. 1. 2. 7. 2. 3. 7. 5. 1. 3.]
RNN Predicted y: [4.614479  4.3671975 4.534924  4.5111637 4.4964504 4.555177  4.5321836
 4.3942285 4.598049  4.4931517]
LSTM Predicted y: [7.853002  1.1469796 3.1693492 3.5572238 3.4494476 3.5780272 4.0179214
 3.7534535 4.370585  3.9545755]
GRU Predicted y: [7.845748   1.220295   2.0333028  6.8348293  1.7372798  3.2303944
 6.318702   5.699797   0.23314214 3.1794047 ]


#### Задание 2 (дополнительное и необязательное)
применить LSTM для решения лекционного практического [задания](colab.research.google.com...mRGTSSeICL)